In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [41]:
#ダミー変数化、欠損値埋め。学習用には,Name,Ticketといった不必要なデータを落とす。Cabinも欠損が多すぎるため削除。
#Ageは平均で埋める。
df = pd.read_csv("train.csv")
y=df["Survived"].as_matrix()
df = df.drop(["Survived","PassengerId","Ticket","Cabin"], axis = 1)
df = df.replace("male",0).replace("female",1)
df = df.replace("S",0).replace("C",1).replace("Q",2)
df = df.fillna(value = {'Embarked': 1})
df = df.fillna(value = {"Age":-1})
honor = []
for i in df["Name"]:
    if "Master" in i :
        honor.append(0)
    elif "Mr" in i :
        honor.append(1)
    elif "Miss" in i :
        honor.append(2)
    elif "Mrs" in i :
        honor.append(3)
    else:
        honor.append("4")
    
df['honor'] = np.array(honor)
a = df.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df[df["honor"]==i[1]]["Age"].mean()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df = df.drop(["Age","honor","Name"], axis = 1)
df = pd.concat([df, frame], axis=1)

In [42]:
df.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age,honor
0,3,0,1,0,7.2500,0.0,22,1
1,1,1,1,0,71.2833,1.0,38,1
2,3,1,0,0,7.9250,0.0,26,2
3,1,1,1,0,53.1000,0.0,35,1
4,3,0,0,0,8.0500,0.0,35,1


In [43]:
X= df.as_matrix()



In [44]:
#ランダムフォレストで学習。
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [45]:
forest = RandomForestClassifier(min_samples_leaf=3, random_state=0)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [46]:
print('Train score: {}'.format(forest.score(X_train, y_train)))
print('Test score: {}'.format(forest.score(X_test, y_test)))

Train score: 0.8877245508982036
Test score: 0.8385650224215246


In [47]:
#勾配ブースティングで学習。
from sklearn.ensemble import GradientBoostingClassifier
gbct = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1)
gbct.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [48]:
print('train accuracy: %.3f' % gbct.score(X_train, y_train))
print('test  accuracy: %.3f' % gbct.score(X_test, y_test))

train accuracy: 0.910
test  accuracy: 0.852


In [54]:
df2 = pd.read_csv("test.csv")
df2 = df2.drop(["PassengerId","Ticket","Cabin"], axis = 1)
df2 = df2.replace("male",0).replace("female",1)
df2 = df2.replace("S",0).replace("C",1).replace("Q",2)
df2 = df2.fillna(value = {'Embarked': 1})
df2 = df2.fillna(value = {"Age":-1})
honor = []
for i in df2["Name"]:
    if "Master" in i :
        honor.append(0)
    elif "Mr" in i :
        honor.append(1)
    elif "Miss" in i :
        honor.append(2)
    elif "Mrs" in i :
        honor.append(3)
    else:
        honor.append("4")
    
df2['honor'] = np.array(honor)
a = df2.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df2[df2["honor"]==i[1]]["Age"].mean()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df2 = df2.drop(["Age","honor","Name"], axis = 1)
df2 = pd.concat([df2, frame], axis=1)

In [55]:
df2 = df2.fillna(df2.mean())
X1= df2.as_matrix()

In [56]:
y1 = gbct.predict(X1)

In [57]:
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y1})
dfnew2.to_csv("pred10.csv") 

In [58]:
y2 = forest.predict(X1)

In [59]:
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y2})
dfnew2.to_csv("pred11.csv") 